In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_data = pd.read_excel("/Volumes/T7 1TB SSD/Empirical_Analysis_FM/Applied_Research_Framework_Git/Applied_projects/Crude_oil_market_analysis/Regression_Urals_loading_analysis_mod.xlsx", sheet_name="Data")
print(df_data)

          Date  LCOc1  NWEMURLCRKMc1   .IMOEX    RSX  BFO-URL-NWE  \
0   2017-01-06  56.75      -0.545276  2213.93  21.52        -2.50   
1   2017-01-13  55.59       1.238238  2195.19  21.45        -2.05   
2   2017-01-20  55.49       0.386498  2159.96  21.08        -1.50   
3   2017-01-27  55.46       0.348593  2266.05  22.08        -1.60   
4   2017-02-03  56.72       0.219638  2226.61  21.80        -1.75   
..         ...    ...            ...      ...    ...          ...   
230 2021-10-22  85.77       7.090000  4196.96  32.67        -1.80   
231 2021-10-29  84.38       3.930000  4150.00  32.00        -1.40   
232 2021-11-05  82.55       1.860000  4174.76  31.93        -1.60   
233 2021-11-12  81.95       1.750000  4121.66  31.27        -1.45   
234 2021-11-19  78.66       0.030000  4016.47  29.92        -1.65   

     Urals loading  
0     1.391864e+07  
1     1.480585e+07  
2     1.377569e+07  
3     1.332605e+07  
4     1.538791e+07  
..             ...  
230   1.347257e+07  
231

In [29]:
real_data = df_data[['Urals loading', 'LCOc1', '.IMOEX', 'NWEMURLCRKMc1']]

print(real_data)

df_data_standardized = (real_data - real_data.mean()) / real_data.std()
df_data_zscore = df_data_standardized[(np.abs(stats.zscore(df_data_standardized)) < 3).all(axis=1)]

     Urals loading  LCOc1   .IMOEX  NWEMURLCRKMc1
0     1.391864e+07  56.75  2213.93      -0.545276
1     1.480585e+07  55.59  2195.19       1.238238
2     1.377569e+07  55.49  2159.96       0.386498
3     1.332605e+07  55.46  2266.05       0.348593
4     1.538791e+07  56.72  2226.61       0.219638
..             ...    ...      ...            ...
230   1.347257e+07  85.77  4196.96       7.090000
231   1.275017e+07  84.38  4150.00       3.930000
232   9.529337e+06  82.55  4174.76       1.860000
233   3.400765e+06  81.95  4121.66       1.750000
234   1.018841e+07  78.66  4016.47       0.030000

[235 rows x 4 columns]


In [30]:
corr_real = df_data_zscore.corr()
print(corr_real)

               Urals loading     LCOc1    .IMOEX  NWEMURLCRKMc1
Urals loading       1.000000  0.229433 -0.409401      -0.023755
LCOc1               0.229433  1.000000  0.291510       0.221110
.IMOEX             -0.409401  0.291510  1.000000       0.218897
NWEMURLCRKMc1      -0.023755  0.221110  0.218897       1.000000


In [31]:
means_real = real_data.mean().values
stds_real = real_data.std().values

In [32]:
cov_synthetic = np.outer(stds_real, stds_real) * corr_real

In [ ]:
# script to generate the synthetic

n_obs = 234
np.random.seed(42)
synthetic_data = np.random.multivariate_normal(
    mean=means_real,
    cov=cov_synthetic,
    size=234)

print(f"✓ generated successfully: shape {synthetic_data.shape}")

✓ generated successfully: shape (234, 4)


In [ ]:
# we verify synthetic data has same correlations as real

df_synthetic = pd.DataFrame(synthetic_data,
columns=['Urals loading', 'LCOc1', '.IMOEX', 'NWEMURLCRMc1'])

print("\nsynthetic correlations:")
print(df_synthetic.corr())

print("real correlations:")
print(corr_real)


synthetic correlations:
               Urals loading     LCOc1    .IMOEX  NWEMURLCRMc1
Urals loading       1.000000  0.228757 -0.424109     -0.078542
LCOc1               0.228757  1.000000  0.228854      0.180391
.IMOEX             -0.424109  0.228854  1.000000      0.173266
NWEMURLCRMc1       -0.078542  0.180391  0.173266      1.000000
real correlations:
            URALS     BRENT      MOEX   NWEMURL
URALS    1.000000  0.229433 -0.409401 -0.023755
BRENT    0.229433  1.000000  0.291510  0.221110
MOEX    -0.409401  0.291510  1.000000  0.218897
NWEMURL -0.023755  0.221110  0.218897  1.000000


In [48]:
print(df_synthetic)

     Urals loading      LCOc1       .IMOEX  NWEMURLCRMc1
0     1.059703e+07  67.238946  2919.941305      7.262149
1     1.284399e+07  80.200164  2789.653172      5.370577
2     1.356745e+07  54.512218  2294.938893     -0.259927
3     1.138023e+07  51.628988  3851.782673      0.681499
4     1.523793e+07  52.545936  2286.288842     -3.381311
..             ...        ...          ...           ...
229   1.025522e+07  53.383874  2765.805059      2.205938
230   8.823373e+06  64.974821  3563.496552      0.628778
231   8.450099e+06  59.487267  3872.741367      4.076034
232   7.311115e+06  50.510020  3148.428918      7.787134
233   1.657584e+07  81.777149  3586.324696      2.117182

[234 rows x 4 columns]


In [49]:
df_synth_standardized = (df_synthetic - df_synthetic.mean()) / df_synthetic.std()
df_synth_variables_zscore = df_synth_standardized[(np.abs(stats.zscore(df_synth_standardized)) < 3).all(axis=1)]

corr_synthetic = df_synth_variables_zscore .corr()
print(corr_synthetic)

               Urals loading     LCOc1    .IMOEX  NWEMURLCRMc1
Urals loading       1.000000  0.226473 -0.443337     -0.077956
LCOc1               0.226473  1.000000  0.216384      0.182516
.IMOEX             -0.443337  0.216384  1.000000      0.181294
NWEMURLCRMc1       -0.077956  0.182516  0.181294      1.000000


In [ ]:
start_date = pd.Timestamp('2017-01-06')
df_synthetic['Date'] = pd.date_range(start=start_date, periods=n_obs, freq='W')

# reorder columns: date first, then data

df_synthetic_data = df_synthetic[['Date','Urals loading', 'NWEMURLCRMc1', '.IMOEX', 'LCOc1']]

In [ ]:
df_synthetic_data.to_csv('urals_synthetic_data.csv', 
                         index=False,           # Don't save index
                         columns=['Date', 'Urals loading', 'NWEMURLCRMc1', '.IMOEX', 'LCOc1'])

# Read back and verify

df_verify = pd.read_csv('urals_synthetic_data.csv')
print("\nColumns AFTER reading CSV:")
print(df_verify.columns.tolist())
print("\nFirst 3 rows:")
print(df_verify.head(3))



Columns AFTER reading CSV:
['Date', 'Urals loading', 'NWEMURLCRMc1', '.IMOEX', 'LCOc1']

First 3 rows:
         Date  Urals loading  NWEMURLCRMc1       .IMOEX      LCOc1
0  2017-01-08   1.059703e+07      7.262149  2919.941305  67.238946
1  2017-01-15   1.284399e+07      5.370577  2789.653172  80.200164
2  2017-01-22   1.356745e+07     -0.259927  2294.938893  54.512218
